In [1]:
import sys
import os
import tensorflow as tf
import matplotlib.pyplot as plt

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from src.training import data_split_augment, train_model
from src.model import *
from src.visualization import *

In [2]:
train_ds, val_ds = data_split_augment("../data/raw", (224,224), 16)

Found 360 files belonging to 9 classes.
Using 288 files for training.
['apple fruit', 'banana fruit', 'cherry fruit', 'chickoo fruit', 'grapes fruit', 'kiwi fruit', 'mango fruit', 'orange fruit', 'strawberry fruit']
Found 360 files belonging to 9 classes.
Using 72 files for validation.
